# Installing the dependencies

In [ ]:
!pip install mediapipe opencv-python

# Importing the libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import math
print("Everything imported succesfully☑️")


Everything imported succesfully☑️


In [2]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Path to your video file
video_path = "Test.mp4"

# Create a VideoCapture object to read from the video file
cap = cv2.VideoCapture(video_path)

with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
    while cap.isOpened():
        success, frame = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Recolor frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make pose detections
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks and render them
        try:
            landmarks = results.pose_landmarks.landmark
            # Render detections
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )
        except:
            pass

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
# Output directory for the processed video
output_dir = "../output"  

video_path = "../Test.mp4"
cap = cv2.VideoCapture(video_path)

In [9]:
# Define video writer (outside the loop for efficiency)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for MP4 format
fps = cap.get(cv2.CAP_PROP_FPS)  # Get original frame rate
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_video = cv2.VideoWriter(f"{output_dir}/processed_video.mp4", fourcc, fps, (frame_width, frame_height))

In [10]:
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose:
    frame_count = 0
    skip_count = 2  # Number of frames to skip between processing

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Process only every skip_count frame
        if frame_count % skip_count == 0:
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks and render them (optional)
            try:
                landmarks = results.pose_landmarks.landmark
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                )
            except:
                pass

            # Write the processed frame to the output video
            out_video.write(image)

        # Display the processed frame (if applicable)
        cv2.imshow('Mediapipe Feed', image)

        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out_video.release()
cv2.destroyAllWindows()

### Landmarks To Consider
- 11 : Left Shoulder
- 12 : Right Shoulder
- 13 : Left elbow
- 14 : Right elbow
- 15 : Left wrist
- 16:  Right wrist

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

In [ ]:
ls_cord = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
rs_cord = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

le_cord = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
re_cord = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

lw_cord = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
rw_cord = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

In [ ]:
def angle_between_points(p1, p2):
    """
    Calculate the angle between two points in 3D space.
    
    Args:
        p1 (Point): First point, with attributes x, y, and z representing its coordinates.
        p2 (Point): Second point, with attributes x, y, and z representing its coordinates.
        
    Returns:
        float: Angle between the two points in degrees.
        
    Notes:
        - Requires the 'math' module.
        - The function uses the dot product of the vectors formed by the two points
          to calculate the angle between them.
    """
    x1, y1, z1 = p1.x, p1.y, p1.z
    x2, y2, z2 = p2.x, p2.y, p2.z
    
    # Calculate the dot product of the two vectors
    dot_product = x1 * x2 + y1 * y2 + z1 * z2
    
    # Calculate the magnitude of each vector
    magnitude1 = math.sqrt(x1 ** 2 + y1 ** 2 + z1 ** 2)
    magnitude2 = math.sqrt(x2 ** 2 + y2 ** 2 + z2 ** 2)
    
    # Calculate the cosine of the angle between the vectors
    cosine_angle = dot_product / (magnitude1 * magnitude2)
    
    # Use arccos to get the angle in radians
    angle_rad = math.acos(cosine_angle)
    
    # Convert radians to degrees
    angle_deg = math.degrees(angle_rad)
    
    return angle_deg

In [ ]:
angle = angle_between_points(ls_cord,rs_cord)
print("Angle between the two points:", angle, "degrees")